<a href="https://colab.research.google.com/github/d61h6k4/notebooks/blob/master/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-datasets matplotlib numpy
!pip install tensorflow==2.0.0-beta1

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [0]:
class DAWNNet(tf.keras.Model):
    def __init__(self):
        super(DAWNNet, self).__init__()
        self.__prep = tf.keras.Sequential(layers=[
            tf.keras.layers.Conv2D(64, 3, strides=1, padding="same", use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU()
        ], name='prep')
        self.__final = tf.keras.Sequential(layers=[
            tf.keras.layers.MaxPool2D(4),
            tf.keras.layers.Flatten()
        ], name='pool')
        self.__classifier = tf.keras.layers.Dense(10, activation='softmax', name='classifier')
        
    def __call__(self, inputs, training=False):
        prepeared_images = self.__prep(inputs, training)
        finalized_images = self.__final(prepeared_images, training)
        images_classes = self.__classifier(finalized_images)
        
        return images_classes

In [0]:
def preprocess_image(example):
    example['image'] = tf.image.transpose(
        tf.image.per_image_standardization(
            tf.pad(tf.cast(example['image'], tf.float32), tf.constant([[4, 4], [4, 4], [0, 0]]), "REFLECT")))
    return example

In [89]:
train, test = tfds.load(name='cifar10', split=['train', 'test'])
train = train.map(preprocess_image).shuffle(buffer_size=1024).batch(64)
test = test.map(preprocess_image)

W0827 17:20:06.367926 140080297170816 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [0]:
model = DAWNNet()

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

for step, example in enumerate(train):
    with tf.GradientTape() as tape:
        logits = model(example['image'], training=True)
        loss = loss_fn(example['label'], logits)
        
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    if step % 100 == 0:
        print(step, float(loss))

W0827 17:22:33.022086 140080297170816 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


0 2.4344582557678223
100 2.1819639205932617
200 2.0250706672668457
300 2.0052566528320312
400 2.0806949138641357
500 1.802386999130249


In [0]:
for example in test:
    pred = model(example['image'])
    print(f'True label is {example["label"]} and predicted one is {pred}')